<a href="https://colab.research.google.com/github/Manuja100/Amazon-Food-Reviews-Sentiment-Analysis/blob/main/Amazon_Food_Reviews_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>